In [1]:
# generate the monomials
varcounter=1; monom_str=[]; onevars=[]; Bohr={1:'1'}
for n in range(1,120):
    if is_prime(n):
        newvar='z'+str(varcounter); varcounter=varcounter+1
        monom_str=monom_str+[newvar]
        onevars=onevars+[newvar]
        Bohr[n]=newvar
    else:
        pfac=list(factor(n))
        newmonom='1'
        for (p,e) in pfac:
            newmonom = newmonom + '*' + Bohr[p] + '^'+ str(e)
        monom_str=monom_str+[newmonom]
#P=PolynomialRing(QQ,onevars)
#print(monom_str)
############
#
# establish extension field and evaluate monomials
#
###########
#S.<sqrt2,sqrt3,sqrt5,sqrt7,sqrt11,sqrt13,sqrt17> = QQ.extension([x^2-2, x^2-3, x^2-5, x^2-7, x^2-11, x^2-13, x^2-17])
#S.<A>=PolynomialRing(QQ)
#assume(A,'rational')
S=QQ
Q.<z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30>=PolynomialRing(S,order='negdeglex')
#sigh, I can't figure out how to make it appreciate the variables
############
varseval=[]
for n in Bohr.keys():
    varseval=varseval+[sage_eval(Bohr[n],locals=vars())]
varseval.remove(1)
print(varseval)
monomslist=[]
for monstr in monom_str:
    monomslist = monomslist + [sage_eval(monstr,locals=vars())]
print(monomslist)

[z1, z2, z3, z4, z5, z6, z7, z8, z9, z10, z11, z12, z13, z14, z15, z16, z17, z18, z19, z20, z21, z22, z23, z24, z25, z26, z27, z28, z29, z30]
[1, z1, z2, z1^2, z3, z1*z2, z4, z1^3, z2^2, z1*z3, z5, z1^2*z2, z6, z1*z4, z2*z3, z1^4, z7, z1*z2^2, z8, z1^2*z3, z2*z4, z1*z5, z9, z1^3*z2, z3^2, z1*z6, z2^3, z1^2*z4, z10, z1*z2*z3, z11, z1^5, z2*z5, z1*z7, z3*z4, z1^2*z2^2, z12, z1*z8, z2*z6, z1^3*z3, z13, z1*z2*z4, z14, z1^2*z5, z2^2*z3, z1*z9, z15, z1^4*z2, z4^2, z1*z3^2, z2*z7, z1^2*z6, z16, z1*z2^3, z3*z5, z1^3*z4, z2*z8, z1*z10, z17, z1^2*z2*z3, z18, z1*z11, z2^2*z4, z1^6, z3*z6, z1*z2*z5, z19, z1^2*z7, z2*z9, z1*z3*z4, z20, z1^3*z2^2, z21, z1*z12, z2*z3^2, z1^2*z8, z4*z5, z1*z2*z6, z22, z1^4*z3, z2^4, z1*z13, z23, z1^2*z2*z4, z3*z7, z1*z14, z2*z10, z1^3*z5, z24, z1*z2^2*z3, z4*z6, z1^2*z9, z2*z11, z1*z15, z3*z8, z1^5*z2, z25, z1*z4^2, z2^2*z5, z1^2*z3^2, z26, z1*z2*z7, z27, z1^3*z6, z2*z3*z4, z1*z16, z28, z1^2*z2^3, z29, z1*z3*z5, z2*z12, z1^4*z4, z30, z1*z2*z8, z3*z9, z1^2*z10, z2^2*z6

In [2]:
#the 'DICT' command I was previously using is outdated now, but I still need to be able to get the degrees
def multideg(monom): #take a monomial in z1 and z2 and return an ordered pair with its multidegree 
                    #(there's probably a way to do this more generally)
    z1deg=monom.degree(z1)
    z2deg=monom.degree(z2)
    return (z1deg,z2deg)
def coeffdict(poly,monomslist,varseval): #takes a polynomial (as an expression) and a pregenerated list of monomials and returns a dictionary
                                # keys are ordered tuples of degrees of a monomial, entries are coefficients
    polydict={(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): poly((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))}
    for monmindex in range(1,len(monomslist)):
        monomattempt=monomslist[monmindex]
        #print(monomattempt)
        multi=tuple(monomattempt.degree(thevar) for thevar in varseval)
        #print(multi)
        d={}
        for n in range(len(varseval)):
            d.update({varseval[n]:multi[n]})
        #print(d)
        polydict.update({multi:poly.coefficient(d)})
    return polydict

In [3]:
############
#
#functions to actually do Gram Schmidt
#
############
def InnerProd(f,g,monomslist,varseval):
    cfsf=coeffdict(f,monomslist,varseval)#the coefficients paired into a dictionary with the multi-indices
    cfsg=coeffdict(g,monomslist,varseval)
    multi_indices = list(set(cfsf.keys()).union(set(cfsg.keys()))) #pretty sure unioning them is unnecessary, but *shrug*
    ip=0
    for alpha in multi_indices:
        if alpha in cfsf.keys():
            term = cfsf[alpha]
        else:
            term=0
        if alpha in cfsg.keys():
            term = term *(cfsg[alpha])
            #print(pairs[m],cfsg[pairs[m]])
        else:
            term=0
        ip=ip+term
    return ip
def wIP(f,g,weight,monomslist,varseval):
    wip=InnerProd(f*weight,g*weight,monomslist,varseval)
    return wip
def wproj(f,g,weight,monomslist,varseval):
    ip=wIP(f,g,weight,monomslist,varseval)
    normsquared=wIP(g,g,weight,monomslist,varseval)
    proj=(ip/normsquared)*g
    return proj
#Gram Schmidt
#given an ordered list of linearly independent basis vectors (monomials in this case),
#and a list of orthogonal polynomials previously computed from the given basis vectors,
# this returns the next one. IT DOES NOT CHANGE THE INPUTTED LISTS. (could probably write a recursive function that would be more efficient, but I'm lazy)
def nextGS(basislist,OGlist,weight,monomslist,varseval):
    error=0
    nextbasis=basislist[len(OGlist)]
    #print "nextbasis=",nextbasis
    for phi in OGlist:
        error=error+wproj(nextbasis,phi,weight,monomslist,varseval)
        #print "error",error
    PHInext=nextbasis-error
    return PHInext
def GS_UpTo(basislist,weight,howmany,monomslist,varseval):
    OGlist=[1]
    for n in range(1,howmany):
        OGlist=OGlist+[nextGS(basislist,OGlist,weight,monomslist,varseval)]
    return OGlist

In [6]:
#Here are the z(lambda-z)^N examples translated to the polydisk with our DS monomial ordering and choosing lambda=L:
N2half=(1/2)^2 - 2*(1/2)*z1 + z2
N2L1=1-2*z1+z2
N2L2=2^2 - 2*2*z1 + z2
N2L3=3^2 - 2*3*z1 + z2
N2L4=4^2 - 2*4*z1 + z2
#
N3half=(1/2)^3 - 3*(1/2)^2*z1 + 3*(1/2)*z2 - z1^2
N3L1=1-3*z1+3*z2-z1^2
N3L2=2^3 - 3*2^2*z1 + 3*2*z2 - z1^2
N3L3=3^3 - 3*3^2*z1 + 3*3*z2 - z1^2
N3L4=4^4 - 3*4^2*z1 + 3*4*z2 - z1^2 
#L^4*z - 4*L^3*z^2 + 6*L^2*z^3 - 4*L*z^4 + z^5
N4L1=1-4*z1+6*z2-4*z1^2+z3    
#
#L^5*z - 5*L^4*z^2 + 10*L^3*z^3 - 10*L^2*z^4 + 5*L*z^5 - z^6
N5L1=1 - 5*z1 + 10*z2 - 10*z1^2 + 5*z3 - z1*z2
#
#L^10*z - 10*L^9*z^2 + 45*L^8*z^3 - 120*L^7*z^4 + 210*L^6*z^5 - 252*L^5*z^6 + 210*L^4*z^7 - 120*L^3*z^8 + 45*L^2*z^9 - 10*L*z^10 + z^11
N10L1=1-10*z1 + 45*z2 - 120*z1^2 + 210*z3 - 252*z1*z2 + 210*z4 - 120*z1^3 + 45*z2^2 - 10*z1*z3 + z5
# here are the ones with a + instead of a -
Np2=1+2*z1+z2
Np3=1+3*z1+3*z2+z1^2
Np4=1+4*z1+6*z2+4*z1^2+z3    
Np5=1 + 5*z1 + 10*z2 + 10*z1^2 + 5*z3 + z1*z2
Nikolski=[N2L1,N3L1,N4L1,N5L1,N10L1]
Nikolskiplus=[Np2,Np3,Np4,Np5]
NikolskiLs=[N2half,N2L1,N2L2,N2L3,N2L4,N3half,N3L1,N3L2,N3L3,N3L4]
for f in NikolskiLs:
    show(f)

1/4 - z1 + z2

1 - 2*z1 + z2

4 - 4*z1 + z2

9 - 6*z1 + z2

16 - 8*z1 + z2

1/8 - 3/4*z1 + 3/2*z2 - z1^2

1 - 3*z1 + 3*z2 - z1^2

8 - 12*z1 + 6*z2 - z1^2

27 - 27*z1 + 9*z2 - z1^2

256 - 48*z1 + 12*z2 - z1^2

In [13]:
alltheOGs=[]
for f in [N4L1,N5L1]:
    show(f)
    #
    DSOGs=[f]
    DSOGs=DSOGs+GS_UpTo(monomslist,f,15,monomslist,varseval)
    alltheOGs=alltheOGs+[DSOGs]
    #
    #
    print("test function")
    print("\\begin{equation*}")
    print("\\scalebox{2}{\\ensuremath{f=",latex(f),"}}")
    print("\\end{equation*}")
    #
    print("\\begin{align*}")
    for deg in range(1,len(DSOGs)):
        print('\\phi_{',deg-1,'}&=',latex(DSOGs[deg]),'\\\\')
        print('%%%%%%%%%%%%%%%%%%%')
    print("\\end{align*}")
    #
    #norms of the OGs
    print("\\begin{align*}")
    for deg in range(2,len(DSOGs)):
        OGnorm=InnerProd(DSOGs[deg]*f,DSOGs[deg]*f,monomslist,varseval)
        print("\\norm{\\phi_{",deg-1,"}}^2&=",latex(OGnorm),'\\\\')
        print('%%%%%%%%%%%%%%%%%%%')
    print("\\end{align*}")
    print("")
    print("")    
    print("")    
    print("")    

1 - 4*z1 + 6*z2 + z3 - 4*z1^2

test function
\begin{equation*}
\scalebox{2}{\ensuremath{f= 1 - 4 z_{1} + 6 z_{2} + z_{3} - 4 z_{1}^{2} }}
\end{equation*}
\begin{align*}
\phi_{ 0 }&= 1 \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 1 }&= -\frac{6}{35} + z_{1} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 2 }&= -\frac{177}{1189} + \frac{438}{1189} z_{1} + z_{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 3 }&= \frac{757}{17914} - \frac{633}{8957} z_{1} + \frac{5643}{17914} z_{2} + z_{1}^{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 4 }&= -\frac{493}{38608} + \frac{629}{19304} z_{1} - \frac{129}{2032} z_{2} + z_{3} + \frac{561}{19304} z_{1}^{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 5 }&= \frac{171944}{2677841} - \frac{531093}{2677841} z_{1} - \frac{18474}{140939} z_{2} + \frac{2568}{140939} z_{3} + \frac{901430}{2677841} z_{1}^{2} + z_{1} z_{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 6 }&= z_{4} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 7 }&= -\frac{19366}{14470685} + \frac{1021683}{57882740} z_{1} - \frac{123093}{1523230} z_{2} + \frac{2098}{761615} z_{3} - \frac{526701}{5788274} z_{1}^{2} + \frac{987051}

1 - 5*z1 + 10*z2 + 5*z3 - 10*z1^2 - z1*z2

test function
\begin{equation*}
\scalebox{2}{\ensuremath{f= 1 - 5 z_{1} + 10 z_{2} + 5 z_{3} - 10 z_{1}^{2} - z_{1} z_{2} }}
\end{equation*}
\begin{align*}
\phi_{ 0 }&= 1 \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 1 }&= -\frac{5}{36} + z_{1} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 2 }&= -\frac{740}{8897} + \frac{1515}{8897} z_{1} + z_{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 3 }&= \frac{35895}{1266034} - \frac{416155}{5064136} z_{1} + \frac{276425}{723448} z_{2} + z_{1}^{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 4 }&= -\frac{15960930}{1066663547} + \frac{65205550}{1066663547} z_{1} - \frac{142666325}{1066663547} z_{2} + z_{3} + \frac{145336275}{1066663547} z_{1}^{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 5 }&= \frac{1779306218}{63172285111} - \frac{5985284685}{63172285111} z_{1} - \frac{14114740805}{126344570222} z_{2} + \frac{7573022965}{126344570222} z_{3} + \frac{17759983845}{126344570222} z_{1}^{2} + z_{1} z_{2} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 6 }&= z_{4} \\
%%%%%%%%%%%%%%%%%%%
\phi_{ 7 }&= \frac{7742134125}{30413434146158} + \frac